<a href="https://colab.research.google.com/github/vieirafrancisco/machine-learning-ufal/blob/master/redes_neurais_artificiais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns